In [1]:
# Python 샘플 코드 #
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# 해당 코드에서 크롤링을 시도할 연도(yyyy00)
this_year = 200900

In [3]:
'''
국토교통부_아파트매매 실거래 상세 자료
활용기간 2023.04.12 까지
LAWD_CD = 11110  # 47290(경산) 지역코드 - 각 지역별 코드 행정표준코드관리시스템(www.code.go.kr)의 법정동코드 10자리 중 앞 5자리
DEAL_YMD = 200601  # 계약월 - 실거래 자료의 계약년월 6자리
numOfRows = 500  # 한 페이지 결과 수
pageNo = 1  # 페이지번호
'''
def get_apt_data(LAWD_CD, DEAL_YMD, serviceKey, numOfRows=10000):
    # 호출하려는 OpenAPI URL 정의
    data_url = "".join(
        ["http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?",
         "serviceKey=", serviceKey, "&LAWD_CD=", str(LAWD_CD), "&DEAL_YMD=", str(DEAL_YMD), "&numOfRows=",
         str(numOfRows)])

    # 정의된 OpenAPI URL을 호출
    data_response = requests.get(data_url)
    data_xml = BeautifulSoup(data_response.text, "xml")
    data_xml_row = data_xml.find_all("item")
    data_xml_row

    # XML에 포함된 항목들을 리스트로 저장
    list_items = ["거래금액", "건축년도", "년", "도로명", "도로명건물본번호코드", "도로명건물부번호코드",
                  "도로명시군구코드", "도로명일련번호코드", "도로명지상지하코드", "도로명코드",
                  "법정동", "법정동본번코드", "법정동부번코드", "법정동시군구코드", "법정동읍면동코드", "법정동지번코드",
                  "아파트", "월", "일", "일련번호", "전용면적", "지번", "지역코드", "층", "해제사유발생일", "해제여부"]

    # 리스트의 리스트 형태로 데이터프레임에 저장할 자료 만들기
    list_dataframe = []

    for n_obs in range(0, len(data_xml_row)):
        list_obs_values = []  # 동일한 관측치의 데이터는 모두 이 리스트에 저장된다
        for i in range(0, len(list_items)):
            # 특정 항목이 XML에 아예 존재하지 않는 경우가 있으므로 try 구문 사용
            try:
                list_obs_values.append(data_xml_row[n_obs].find(list_items[i]).text)
            except Exception as e:
                list_obs_values.append("")
                print("Error in " + str(n_obs) + "th obs, item: " + list_items[i] + ", " + str(e))
        list_dataframe.append(list_obs_values)  # 리스트의 리스트로 관측치 저장

    # 데이터프레임 형태로 변환
    df_apt = pd.DataFrame(list_dataframe, columns=list_items)
    return df_apt

In [4]:
# 함수 작동 여부 확인
serviceKey_YKim = "%2F2a7hudNIqdsS1Uq7SaicUQYvpno8zZ4M53iUr4PgZnXIDq4SMhbDLNHrk6PfFXNdnb7LuNpEffhq8YXRq2dbQ%3D%3D"
df_tmp1 = get_apt_data(LAWD_CD=11110, DEAL_YMD=202102, serviceKey=serviceKey_YKim)

Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 25th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 일련번호, 'NoneType' object has no attribute 'text'


In [5]:
# Import TXT file 법정동코드 전체자료 불러오기
#df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
#paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd = pd.read_csv('C:/Users/a/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd["LAWD_CD"] = df_lawd_cd["법정동코드"].divmod(100000)[0]
df_lawd_cd_nodup = df_lawd_cd.drop_duplicates(subset=['LAWD_CD']).copy()  # 중복값 제외
df_lawd_cd_nodup_exist = df_lawd_cd_nodup[df_lawd_cd_nodup["폐지여부"] == "존재"].copy()

In [6]:
total_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"]
tmp_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"][3:10]

In [7]:
# 국토교통부_아파트매매 실거래 상세 자료는 2008년 1월부터 자료가 존재한다.
# 그런데 2006년으로 입력해도 자료가 있다.

In [8]:

# 시작 시점과 종료 시점 입력
yyyymm_start = this_year + 1  # 시작 년월
yyyymm_end = this_year + 3  # 종료 년월

list_yyyymm = []
for n_yyyymm in range(yyyymm_start, yyyymm_end+1):
    if (divmod(n_yyyymm, 100)[1] != 0) & (divmod(n_yyyymm, 100)[1] < 13):
        list_yyyymm.append(n_yyyymm)

list_yyyymm.reverse()


# StopWatch: 코드 시작
time_this_code_start = datetime.now()
print("This code started at: " + str(time_this_code_start))

# DataFrame 구성을 위해 공백 자료를 하나 만든다.
df_dataset_null = get_apt_data(LAWD_CD=11120, DEAL_YMD=202102, serviceKey=serviceKey_YKim)

# 전국의 한달 자료 받는데 약 9분 소요
for yyyymm in list_yyyymm:
    df_dataset = df_dataset_null.copy()
    for lawd_cd in total_lawd_cd:
        df_apt = get_apt_data(LAWD_CD=lawd_cd, DEAL_YMD=yyyymm, serviceKey=serviceKey_YKim)
        # df_apt.to_pickle("./data_raw/df_apt_" + str(yyyymm) + "_" + str(lawd_cd) + ".pkl")
        df_dataset = pd.concat([df_dataset, df_apt])
    df_dataset.to_pickle("C:/Users/a/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_" + str(yyyymm) + ".pkl")

# StopWatch: 코드 종료
time_this_code_end = datetime.now()
print("This code finished at: " + str(time_this_code_end))
print("Elapsed (in this code): " + str(time_this_code_end - time_this_code_start))

This code started at: 2021-05-19 01:26:46.915922
Error in 24th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 28th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 33th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 34th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 49th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 50th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 53th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 59th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 61th o

Error in 54th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 55th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 55th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 60th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 60th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 75th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 75th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 77th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 77th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 78th obs, item: 도로명건물본번호코드, 'NoneType' obje

Error in 104th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 117th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 118th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 128th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 132th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 134th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 140th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 144th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 145th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 151th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 152th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 157th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 167th obs, item: 도로명지상지하코드, 'NoneType' obje

Error in 495th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 43th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 51th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 246th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 257th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 2th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 4th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 8th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 51th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no attri

Error in 90th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 98th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 99th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 109th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 113th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 115th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 120th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 158th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 165th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 166th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 169th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 182th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 183th obs, item: 도로명지상지하코드, 'NoneType' object 

...

Error in 48th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명일련번호코드, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 71th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 100th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 101th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 105th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 165th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute

Error in 252th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 254th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 255th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 259th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 261th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 262th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 266th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 269th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 270th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 271th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 272th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 277th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 278th obs, item: 도로명지상지하코드, 'NoneType' obje

Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 14th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 26th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 108th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 109th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 110th obs, item: 도로명지상지하코드, 'NoneType' object has no 

Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 2th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 4th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 8th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 9th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 10th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 12th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 13th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 16th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute

Error in 56th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 106th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 12th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 14th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 34th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 7th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 8th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 22th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 23th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 94th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 99th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 104th obs, item: 도로명지상지하코드, 'NoneType' object has no att

In [9]:
# Check the data

data1 = pd.read_pickle('C:/Users/a/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start)+'.pkl')

data2 = pd.read_pickle('C:/Users/a/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 1)+'.pkl')

data3 = pd.read_pickle('C:/Users/a/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 2)+'.pkl')

In [10]:
data1

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"98,145",2008,2009,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),1,9,11110-2203,151.81,9,11110,14,,
1,"82,236",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),1,9,11110-2204,153.42,9-1,11110,11,,
2,"82,236",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),1,9,11110-2204,153.42,9-1,11110,11,,
3,"82,236",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),1,9,11110-2204,153.42,9-1,11110,10,,
4,"82,236",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),1,9,11110-2204,153.42,9-1,11110,9,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,"3,550",1996,2009,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,1,8,50130-44,40,1323-2,50130,2,,
98,"3,550",1996,2009,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,1,8,50130-44,40,1323-2,50130,3,,
99,"3,550",1996,2009,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,1,8,50130-44,40,1323-2,50130,3,,
100,"3,550",1996,2009,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,1,13,50130-44,40,1323-2,50130,3,,


In [11]:
data2

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"124,000",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),2,4,11110-2204,144.52,9-1,11110,13,,
1,"59,000",2008,2009,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),2,9,11110-2203,94.28,9,11110,5,,
2,"64,000",2008,2009,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),2,13,11110-2203,94.51,9,11110,5,,
3,"82,236",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),2,25,11110-2204,153.42,9-1,11110,10,,
4,"82,236",2008,2009,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),2,25,11110-2204,153.42,9-1,11110,11,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,"4,900",1990,2009,마소물로8번길,00010,00001,50130,01,0,4850222,...,로얄하이츠맨션,2,17,50130-4,83.7,875,50130,1,,
28,"5,725",2003,2009,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,2,5,50130-61,59.97,1789,50130,5,,
29,"9,400",2006,2009,신영로,00016,00000,50130,01,0,3350186,...,벽성뜨래별,2,27,50130-517,81.031,1187-2,50130,6,,
30,"4,250",2001,2009,서성일로,01186,00006,50130,01,0,3350145,...,"진우파크빌(101,102동)",2,3,50130-46,59.8944,1618-1,50130,3,,


In [12]:
data3

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"40,200",2002,2009,필운대로10길,00016,00000,11110,01,0,4100521,...,신현(101동),3,25,11110-7,84.82,6-13,11110,3,,
1,"114,000",2008,2009,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),3,9,11110-2203,159.01,9,11110,4,,
2,"105,000",2008,2009,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),3,26,11110-2203,158.99,9,11110,11,,
3,"49,000",1994,2009,새문안로3길,00030,00000,11110,02,0,4100158,...,세종로대우,3,11,11110-14,100.44,167,11110,15,,
4,"123,000",2004,2009,새문안로3길,00023,00000,11110,02,0,4100158,...,경희궁의아침4단지,3,25,11110-117,124.17,73,11110,15,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,"5,000",2003,2009,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,3,20,50130-61,51.88,1789,50130,3,,
30,"5,000",2003,2009,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,3,23,50130-61,51.88,1789,50130,3,,
31,"6,300",2003,2009,천제연로,00239,00000,50130,01,0,3350258,...,중문푸른마을,3,24,50130-61,51.88,1789,50130,3,,
32,"9,300",2006,2009,신영로,00016,00000,50130,01,0,3350186,...,벽성뜨래별,3,6,50130-517,81.031,1187-2,50130,7,,
